Install all the dependencies

In [ ]:
install.packages("twitteR",repos="http://cran.us.r-project.org")
install.packages("ggmap",repos="http://cran.us.r-project.org")
install.packages("hash",repos="http://cran.us.r-project.org")

In [4]:
## Because of the ggmap library, suppress the warning messages
options(warn=-1)
sink("log.out")
library(twitteR)
library(ggmap)
library(ggplot2)
library(hash)

In [2]:
setup_twitter_oauth("","","","")

[1] "Using direct authentication"


Execute the cell below to download tweets related to a topic and convert them into data frames

In [3]:
#Downlaod tweets
 topic <- '#nfl'
 tweets <- searchTwitter(topic, n=1000)
#Covert into data frames
 tweets_df <- twListToDF(tweets)


Group screen names becuase location is to be determined via screen name

In [5]:
# UserName to Count map
# I asked a question related to the code below @ https://stackoverflow.com/questions/42238057/has-key-method-in-hash-package-of-r-not-storing-string-key-properly
# Please have a look 
user_count_map <- hash()
 i=1
 num_tweets <- nrow(tweets_df)
 while(i<= num_tweets){
 screen_name <- tweets_df[i,]$screenName
 if(has.key(screen_name,user_count_map)==TRUE){
    count <- user_count_map[[screen_name]]
    user_count_map[[screen_name]] <- (count +1)
} else {
   user_count_map[[screen_name]] <- 1
}
i = i+1
}

Get All Locations and get location to count mapping. This way the number of unique locations would reduce and hence lesser number of API calls would be required 

In [6]:
# Get All Locations and get location to count mapping
 all_users <- lookupUsers(keys(user_count_map))
 location_count_map <- hash()
 for(u in all_users){
location <- u$location
screenName <- u$screenName
 if(!is.null(location) && location!=""){
if((has.key(location,location_count_map))==TRUE){
    location_count_map[[location]] <- location_count_map[[location]] + user_count_map[[screenName]]
} else {
 location_count_map[[location]] <- user_count_map[[screenName]]
} 
}
i=i+1
}

Get all the geocodes to count mapping. Only take those geocodes which falls in US. To check if the location is in US, search for the reverse geocode check if the address ends with "USA". This step may take a long time 

In [ ]:
# Get all the geocodes to count mapping. Only take those geocodes which falls in US
# DIscard all locations which donot fall within US boundaries
reg <- "USA$"
allLocs <- keys(location_count_map)
lat_lon_count_map <- hash()
for(loc in allLocs){
    code <- geocode(loc)
    if(!is.na(code$lon)){
        rev_loc <- c(code$lon,code$lat)
        rev <- revgeocode(rev_loc)
        if(any(grep(reg,rev))){
            str <- c(code$lon,",",code$lat)
            str_2 <- paste(str,collapse='')
            if(has.key(str_2,lat_lon_count_map)){
                lat_lon_count_map[[str_2]] <- lat_lon_count_map[[str_2]] + location_count_map[[loc]]
            } else {
                lat_lon_count_map[[str_2]] <- location_count_map[[loc]]
            }
            #print("Correct")
        } else {
            #print("Correct but irrelevant")
            }
    } else {
        #print("Google couldn't give Lat/Long for this address")
    }
}

locationKeys = keys(lat_lon_count_map)
#values <- values(lat_lon_count_map)
#values
write("",file=topic,append=FALSE,sep='')
for(location in locationKeys){
    count <- lat_lon_count_map[[location]]
    write(location,file=topic,append=TRUE,sep='\n')
    write(count,file=topic,append=TRUE,sep='\n')
}



If you already have a file in specific format, skip the above steps and read the file. 

**** Important ****
If you want to plot fresh data, please use the file name as 'coord_values' ie replace topic with 'coord_values' (including the single quotes)

In [23]:
# Read Co-ordinates from file
coord_data <- read.table(topic)

To give more significance to locations from where more tweets were obtained, get the maximum count among all the locations and set the size and alpha of 

In [24]:
length <- nrow(coord_data)
# get max count
max <- as.numeric(0)
i=2
#as.numeric(as.character(coord_data[[1]][4]))
while(i<=length){
    count <- as.numeric(as.character(coord_data[[1]][i]))
    #print(count)
    if(count>max){
        max<-count
    }
    i=i+2
    }


[1] -104.990251,39.7392358
132 Levels: -101.8551665,33.5778631 ... 9

In [ ]:
# Plot
i = 1
usMap <- get_map(zoom=4)
usMap <- ggmap(usMap)
print(max)
while(i<=length-1){
    coord_loc <- coord_data[[1]][i]
    count <- as.numeric(as.character(coord_data[[1]][i+1]))
    #print(count)
    a <- count/max
    if(a>.5){
        a=.5
    }
    s<- (count/max)*2
    inp <- as.character(coord_loc)
    inp <- strsplit(inp,',')
    gd <- data.frame(lat=c(as.numeric(inp[[1]][2])),lon=c(as.numeric(inp[[1]][1])))
    usMap <- usMap + geom_point(data=gd,aes(x=lon, y=lat),size=s+1,alpha=.5 + a) 
    i=i+2
    
}
ggplot_build(usMap)